<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

In [11]:
import pandas as pd
import folium
from folium import Map, Marker, PolyLine, features, RegularPolygonMarker, DivIcon
from folium.plugins import PolyLineTextPath
import math
import networkx as nx
import time
import random
import multiprocessing

In [ ]:
# 2.3.1

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep = "\t")
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep = "\t")

charging_station_nodes = {323346405, 87120378, 2374444198, 1345740157, 2351742223}

coords = {
    row['osmid']: (row['y'], row['x'])  # Folium notar (lat, lon)
    for _, row in nodes.iterrows()
}

# búum til graf með stefnu og þyngd fyrir edges
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['length'])

## Need to reverse in some cases because it's directed
G_rev = G.reverse()

In [13]:
# 2.3.2

def shortest_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.dijkstra_path_length(graph, source=node_id, target=charger_id, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

In [14]:
# 2.3.3

def process_node_wrapper(args):
    row, charger_nodes, G, charger_only = args
    node_id = row['osmid']
    if node_id in charger_nodes:
        color = "orange"
        popup_text = f"🔌 Hleðslustöð <br>Node ID: {node_id}"
    elif charger_only == False:
        color = "red" if row['primary'] else "blue"
        distance, closest = shortest_distance_to_charger(node_id, G, charger_nodes)
        popup_text = (f"🚗 Node ID: {node_id}<br>Primary: {row['primary']}<br>"
                      f"Fjarlægð frá næstu hleðslustöð: {distance:.2f} meters<br>"
                      f"Næsta hleðslustöð: {closest}")
    else:
        return None
    
    return {
        'location': [row['y'], row['x']],
        'color': color,
        'popup_text': popup_text
    }

# keyrsla gæti tekið sirka 5-10 mín
def create_map(charger_nodes, map_name, chargers_only = False):
    center_lat = nodes['y'].mean()
    center_lon = nodes['x'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    start_time = time.time()

    row_args = [
        (row, charger_nodes, G, chargers_only)
        for _, row in nodes.iterrows()
    ]

    # ryzen 7 go brrrr
    with multiprocessing.Pool() as pool:
        marker_data_list = pool.map(process_node_wrapper, row_args)

    for md in marker_data_list:
        if md is None:
            continue
        
        folium.CircleMarker(
            location=md['location'],
            radius=4,
            color=md['color'],
            fill=True,
            fill_color=md['color'],
            fill_opacity=0.8,
            popup=folium.Popup(md['popup_text'], max_width=250)
        ).add_to(m)

    print(f"Keyrsla tók {int(time.time() - start_time)}s")

    for _, row in edges.iterrows():
        if row['u'] in coords and row['v'] in coords:
            start = coords[row['u']]
            end = coords[row['v']]
            PolyLine(
                locations=[start, end],
                color='gray',
                weight=1,
                opacity=0.5,
                popup=row.get('name', '')
            ).add_to(m)

            # Reiknum gráðu á ör-iconinu og setjum svo á endann á línunni
            # vantar að laga betur, gerir kortið mjög hægt líka.
            """angle = calculate_angle(start, end)
            folium.map.Marker(
                location=end,
                icon=DivIcon(
                    icon_size=(150, 36),
                    icon_anchor=(7, 20),
                    html=f'<div style="transform: rotate({angle}deg); color: gray; font-size: 16px;">&#8594;</div>'
                )
            ).add_to(m)"""

    m.save(map_name)
    print(f"Kort geymt i skra: {map_name}")

create_map(charging_station_nodes, "kort_2_3_3.html")

Keyrsla tók 25s
Kort geymt i skra: kort_2_3_3.html


In [15]:
# 2.3.4

## it's in the code above

In [16]:
# 2.3.5

## Heuristic for A*
## Needlessly complicated on such a small scale
## Calculate the distance between two points on the surface of a sphere
## It's somewhat off up here since the Earth isn't a perfect sphere

def calc_spherical_distance(node_id, target_id):
    lat_source,lon_source = coords[node_id]
    lat_target,lon_target = coords[target_id]

    radius = 6373.0

    delta_lat = math.radians(lat_target) - math.radians(lat_source)
    delta_lon = math.radians(lon_target) - math.radians(lon_source)

    a = math.sin(delta_lat / 2)**2 + math.cos(lat_source) * math.cos(lat_target) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000 #metres

    return distance

def astar_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.astar_path_length(graph, source=node_id, target=charger_id, heuristic=calc_spherical_distance, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

## Return the sum for later use
def process_astar_row(args):
    row, charging_station_nodes, G = args
    node_id = row['osmid']

    distance, closest = astar_distance_to_charger(node_id, G, charging_station_nodes)
    return distance if distance != float("inf") else 0.0

def run_astar(charging_station_nodes, benchmark):
    distance_sum = 0.0
    if benchmark:
        start_time = time.time()

    row_args = [(row, charging_station_nodes, G) for _, row in nodes.iterrows()]
    
    with multiprocessing.Pool() as pool:
        distances = pool.map(process_astar_row, row_args)

    distance_sum = sum(distances)

    if benchmark:
        print(f"Keyrsla tók {int(time.time() - start_time)}s")

    return distance_sum

run_astar(charging_station_nodes, True)

Keyrsla tók 19s


52496231.653249905

In [17]:
# 2.3.6

## For all primary nodes, run Dijkstra on the reversed graph
def simple_dijkstra(charger_id):
    distances = nx.single_source_dijkstra_path_length(G_rev, charger_id, weight='weight')
    distance_sum = sum(distances.values())

    return distance_sum

def place_optimal_charger(primary_nodes):
    best_distance = float("inf")
    best_node = None

    # don't need threads anymore
    for node in primary_nodes:
        distance = simple_dijkstra(node.osmid)

        if distance < best_distance and distance != 0:
            best_distance = distance
            best_node = node

    return best_node, best_distance

primary_nodes = []
for _, row in nodes.iterrows():
    if row['primary'] == True:
        primary_nodes.append(row)

optimal_node, distance = place_optimal_charger(primary_nodes)
print(f"Best node for k = 1 was {optimal_node} with a distance of {distance}")

Best node for k = 1 was osmid       34827739
x         -21.845736
y          64.114075
primary         True
Name: 589, dtype: object with a distance of 71813532.58901912


In [ ]:
# 2.3.7

## Always add the new best node. Obviously have to remove the previously best node

def greedy_k_chargers():
    best_nodes = []

    start_time = time.time()
    for k in range(10):
        best_node, dist = place_optimal_charger(primary_nodes)
        print(f"k = {k+1} Added new best node {best_node} with a distance of {dist}")
        best_nodes.append(best_node)
        # Python is awful btw
        primary_nodes[:] = [node for node in primary_nodes if not node.equals(best_node)]

    print(f"Keyrsla tók {int(time.time() - start_time)}s")
    return best_nodes

greedy_nodes = greedy_k_chargers()
create_map([node.osmid for node in greedy_nodes], "kort_2_3_7.html", True)

k = 1 Added new best node osmid       34827739
x         -21.845736
y          64.114075
primary         True
Name: 589, dtype: object with a distance of 71813532.58901912
k = 2 Added new best node osmid      286837791
x         -21.845392
y          64.113908
primary         True
Name: 2540, dtype: object with a distance of 71875798.64648832
k = 3 Added new best node osmid      470319622
x         -21.846032
y          64.113726
primary         True
Name: 4186, dtype: object with a distance of 71910266.57909779
k = 4 Added new best node osmid      1340963771
x          -21.845933
y           64.114072
primary          True
Name: 7495, dtype: object with a distance of 71918846.18457243
k = 5 Added new best node osmid       76001341
x         -21.845442
y           64.11398
primary         True
Name: 1247, dtype: object with a distance of 71967058.24839959
k = 6 Added new best node osmid      286838075
x         -21.840445
y          64.113323
primary         True
Name: 2542, dtype: obj

In [ ]:
# keyrsla gæti tekið allt að klukkustund mín

# Lesum inn gögnin
nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep="\t")
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep="\t")

# Búum til graf
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['length'])

# Finna primary hnúta
primary_nodes = nodes[nodes["primary"] == True]["osmid"].tolist()

# Hnit hnútanna
coords = {row['osmid']: (row['y'], row['x']) for _, row in nodes.iterrows()}


# Reiknar heildarkostnað F(v1, ..., vk)
def compute_total_cost(graph, chargers):
    cost = 0
    for node in graph.nodes:
        min_dist = float("inf")
        for charger in chargers:
            try:
                d = nx.dijkstra_path_length(graph, source=node, target=charger, weight='weight')
                min_dist = min(min_dist, d)
            except:
                continue
        cost += min_dist
    return cost


# Reikna evklíðska fjarlægð á milli hnúta
def euclidean_distance(coord1, coord2):
    return ((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)**0.5

# Skilar 2 bestu hnútum sem eru staðsettir langt í burtu
def get_top_2_diverse_nodes(graph, selected, candidates, coords):
    scores = []
    for node in candidates:
        trial = selected + [node]
        cost = compute_total_cost(graph, trial)
        scores.append((node, cost))

    # Finna bestu hnútana (lægra cost)
    scores.sort(key=lambda x: x[1])
    top = scores[:5]  # skoðum efstu 5 til að finna tvo sem eru langt í burtu

    # Finna 2 sem eru með mesta fjarlægð á milli sín
    best_pair = (top[0][0], top[1][0])
    max_dist = 0
    for i in range(len(top)):
        for j in range(i + 1, len(top)):
            dist = euclidean_distance(coords[top[i][0]], coords[top[j][0]])
            if dist > max_dist:
                max_dist = dist
                best_pair = (top[i][0], top[j][0])

    return best_pair

# Endurkvæm gráðug leit
def recursive_greedy(graph, selected, candidates, coords, k):
    if len(selected) == k:
        return selected, compute_total_cost(graph, selected)

    best_nodes = []
    best_cost = float("inf")

    remaining = set(candidates) - set(selected)

    # Veljum 2 bestu möguleika sem eru langt í burtu
    if len(remaining) >= 2:
        n1, n2 = get_top_2_diverse_nodes(graph, selected, remaining, coords)
        for next_node in [n1, n2]:
            new_selected = selected + [next_node]
            result_nodes, cost = recursive_greedy(graph, new_selected, candidates, coords, k)
            if cost < best_cost:
                best_cost = cost
                best_nodes = result_nodes
    elif len(remaining) == 1:
        only_node = list(remaining)[0]
        new_selected = selected + [only_node]
        return new_selected, compute_total_cost(graph, new_selected)

    return best_nodes, best_cost

# Yfirlitsfall: keyrir marga random starta
def improved_recursive_greedy_k10(graph, candidates, coords, trials=5):
    best_overall = []
    best_cost = float("inf")
    start_time = time.time()

    for i in range(trials):
        print(f"Tilraun {i+1}/{trials}")
        start_node = random.choice(candidates)
        selected, cost = recursive_greedy(graph, [start_node], candidates, coords, k=10)
        print(f"  → Lausn með start {start_node}, kostnaður: {cost:.2f}")

        if cost < best_cost:
            best_cost = cost
            best_overall = selected

    runtime = time.time() - start_time
    print(f"\nBesti kostnaður: {best_cost:.2f}, Tími: {runtime:.2f} sekúndur")
    return best_overall, runtime

# Teiknar kort með 10 stöðvum (sleppum að sýna alla hnúta hér því að þá mundi keyrslan vera miklu lengri og kortið töluvert hægara)
def render_chargers_map(selected_nodes, coords):
    m = folium.Map(location=[64.1355, -21.8954], zoom_start=12)

    for node_id in selected_nodes:
        latlon = coords[node_id]
        popup = f"Hleðslustöð (valin)<br>Node ID: {node_id}"

        folium.CircleMarker(
            location=latlon,
            radius=6,
            color="orange",
            fill=True,
            fill_color="orange",
            fill_opacity=0.9,
            popup=popup
        ).add_to(m)

    m.save("238kort.html")
    print("Kort vistað sem 238kort.html")

# Keyrsla
best_nodes, total_time = improved_recursive_greedy_k10(G, primary_nodes, coords, trials=10)
render_chargers_map(best_nodes, coords)